# Dudas y preguntas frecuentes

### Las ventanas no muestran las imágenes

Hace falta `cv.waitKey(ms)` para [refrescar el interfaz](https://docs.opencv.org/4.1.0/d7/dfc/group__highgui.html#ga5628525ad33f52eab17feebcfba38bd7) de opencv. Esto se hace automáticamente dentro del bucle de captura de autoStream(). En cualquier otro caso es necesario llamar a esta función para que se refresquen la ventanas aunque no necesitemos la tecla pulsada.

### Un módulo no tiene las funciones que debería tener

Si un archivo del directorio de trabajo tiene el mismo nombre que un módulo del sistema (por ejemplo "numpy.py", "dlib.py", etc.), un import de ese módulo producirá un error: Python cargará el archivo local en lugar del módulo y no encontrará sus funciones. Simplemente hay que tener cuidado con los nombres de archivo de nuestro código fuente para que no coincidan con ningún módulo.

### Ejercicio CLASIFICADOR

El modelo reconocido puede indicarse mostrándolo en pequeño en una esquina de la ventana.

Se puede añadir la opción de guardar modelos sobre la marcha al pulsar una tecla.

### Ejercicio SIFT

Algunas recomendaciones:

Los objetos deben tener "detalles" para que aparezcan dentro suficientes puntos de interés. Hay objetos muy uniformes que producen muy pocos puntos, o salen casi todos por los bordes o el exterior, y entonces no son muy apropiados para este método. Por ejemplo una libreta con tapas negras, o un teléfono móvil, una botella de cristal sin etiqueta, etc. no funcionan bien.

Hay que calcular una sola vez los keypoints y descriptores de los modelos. En el bucle de captura solo deben calcularse los de la imagen actual.

Para reducir el tiempo de cálculo se puede trabajar con imágenes me menor resolución (eg. 400x300) y limitar el número de keypoints en las comparaciones (parámetro nfeatures). 

Si usando autoStream() se produce un retardo grande de la imagen en vivo, puede ser conveniente hacer captura asíncrona como se explica en los ejemplos de `code/thread`.

Sí a pesar de todo el proceso va muy lento, se puede efectuar la extracción de keypoints y la comparación con los modelos solo cuando detectemos que la imagen está bastante quieta o cuando pulsemos una tecla.

Si los modelos tienen diferente número de keypoints la comparación debe hacerse teniendo en cuenta el porcentaje de coincidencias, no el valor absoluto.

Se puede rechazar la decisión cuando el porcentaje ganador sea pequeño, cuando el segundo mejor sea parecido al primero, o cuando haya pocas coincidencias en la imagen, entre otras situaciones que dependen de la aplicación.

Cuando no se detecta ningún keypoint los descriptores no se devuelven como un array de dimension 0x128, sino como un valor `None`. Hay que tener cuidado con esto para que no se produzcan errores en tiempo de ejecución. Esto puede ocurrir cuando la cámara apunta hacia la mesa, o está muy desenfocada.

Aquí hay un [vídeo de ejemplo](https://robot.inf.um.es/material/va/sift-demo.mp4) de lo que se puede conseguir sin muchas complicaciones. Se muestra en pequeño el modelo ganador, su porcentaje, y la diferencia con el segundo mejor. Observa que los vinilos se reconocen aunque no se vean enteros en la imagen, con diferentes tamaños, con cualquier rotación en el plano de imagen, y con cierta inclinación de perspectiva. Hay también cierta resistencia al desenfoque y a reflejos. (Aunque no se ven en esta breve secuencia, pueden producirse clasificaciones erróneas cuando la escena tiene muchos puntos y no hay ningún modelo conocido.)

### Ejercicio VROT 

En este ejercicio lo más sencillo es partir de `code/lk_tracks.py` y reducir la longitud de los tracks para que sus longitudes se aproximen más al movimiento instantáneo. El siguiente pantallazo muestra el efecto de un giro de la cámara hacia la izquiera y ligeramente hacia abajo.

![vrot](../images/demos/vrot.png)

Los movimientos de giro lateral "[pan][pan]" (izquierda-derecha) y "[tilt][tilt]" (arriba-abajo) son fáciles de deducir con el valor medio de los desplazamientos. Lo dibujamos desde el centro de la imagen, ampliado para que vea mejor y en dirección opuesta (la del giro de la cámara).

[pan]:https://en.wikipedia.org/wiki/Panning_(camera)

[tilt]: https://en.wikipedia.org/wiki/Tilt_(camera)

Este desplazamiento medio en los últimos $n$ frames (la longitud de los tracks), en pixels/frame, se puede convertir a grados/segundo teniendo en cuenta los fps de captura y el FOV de la cámara. En este ejercicio no buscamos mucha precisión. Lo importante es obtener el orden de magnitud correcto de las medidas.

Para detectar el movimiento de avance/retroceso podemos comparar el campo de velocidades con vectores de referencia radiales desde el centro. Pero para que esta idea no funcione del todo mal es conveniente restar a los vectores de movimiento observados el movimiento medio. En los pantallazos siguientes se muestran en rojo los vectores de movimiento observados, en amarillo los vectores de movimiento residual al quitar el movimiento medio, en verde los vectores radiales de referencia de un movimiento de avance, y en azul los vectores tangenciales de referencia de un giro alrededor del eje óptico.

![vrot](../images/demos/vrot2.png)

En el primer caso detectamos que además del movimiento lateral hacia abajo hay un movimiento residual (vectores amarillos) que coincide bastante con los vectores de referencia radiales (verdes), indicando que la cámara se mueve hacia delante.

![vrot](../images/demos/vrot3.png)

En el segundo caso el movimiento residual se alinea bastante con los vectores de referencia tangenciales (azules), indicando que la cámara estaba rotando sobre su eje.